In [1]:
!pip install mne --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
#imports
import os
import numpy as np
import mne

In [4]:
stage_map = {
    "W":0,
    "1":1,
    "2":2,
    "3":3,
    "4":3,
    "R":4
}

In [5]:
psg_path = "drive/MyDrive/sleep-edf/sleep-telemetry/ST7011J0-PSG.edf"
raw = mne.io.read_raw_edf(psg_path, preload=True, stim_channel=None, verbose=False)

print("Channel names:", raw.ch_names)
print(raw.info)

/tmp/ipython-input-5-1316513360.py:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, stim_channel=None, verbose=False)
/tmp/ipython-input-5-1316513360.py:2: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, stim_channel=None, verbose=False)


Channel names: ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'EMG submental', 'Marker']
<Info | 8 non-empty values
 bads: []
 ch_names: EEG Fpz-Cz, EEG Pz-Oz, EOG horizontal, EMG submental, Marker
 chs: 5 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 800.0 Hz
 meas_date: 1994-07-12 23:00:00 UTC
 nchan: 5
 projs: []
 sfreq: 100.0 Hz
 subject_info: <subject_info | his_id: X, sex: 1, first_name: Male, last_name: 60yr>
>


In [6]:
def load_and_segment(path_psg, path_hyp, subject_id):
    raw = mne.io.read_raw_edf(path_psg, preload=True, stim_channel=None, verbose=False)

    eeg_channels = [name for name in raw.ch_names if "EEG" in name]
    eog_channels = [name for name in raw.ch_names if "EOG" in name]
    emg_channels = [name for name in raw.ch_names if "EMG" in name]

    sig_eeg = raw.get_data(picks=eeg_channels)
    sig_eog = raw.get_data(picks=eog_channels) if eog_channels else np.zeros((0, raw.n_times))
    sig_emg = raw.get_data(picks=emg_channels) if emg_channels else np.zeros((0, raw.n_times))

    ann = mne.read_annotations(path_hyp)

    Xe, Xo, Xm, labs, subs = [], [], [], [], []

    epoch_len = int(30 * raw.info["sfreq"]) # 30 second windows
    for onset, desc in zip(ann.onset, ann.description):
        char = desc.split()[-1]
        if char in stage_map: # skipping M
            start = int(onset * raw.info["sfreq"])

            if start + epoch_len <= sig_eeg.shape[1]:
                seg_eeg = sig_eeg[:, start:start + epoch_len]
                seg_eog = sig_eog[:, start:start + epoch_len] if sig_eog.size > 0 else np.zeros((0, epoch_len))
                seg_emg = sig_emg[:, start:start + epoch_len] if sig_emg.size > 0 else np.zeros((0, epoch_len))

                Xe.append(seg_eeg.T)
                Xo.append(seg_eog.T)
                Xm.append(seg_emg.T)
                labs.append(stage_map[char])
                subs.append(subject_id)

    return np.array(Xe), np.array(Xo), np.array(Xm), np.array(labs), np.array(subs)

In [8]:
base_dir = "/content/drive/My Drive/sleep-edf"
data_dir = base_dir
all_Xe, all_Xo, all_Xm, all_y,all_subj = [], [], [], [], []

for subset in ["sleep-cassette","sleep-telemetry"]:
    subdir = os.path.join(data_dir, subset)
    for fname in os.listdir(subdir):
        if fname.endswith("PSG.edf"):
            psg_path = os.path.join(subdir, fname)
            prefix = fname.replace("-PSG.edf", "")
            hyp_files = [f for f in os.listdir(subdir) if f.startswith(prefix[:-1]) and f.endswith("Hypnogram.edf")]

            if not hyp_files:
                continue

            hyp_path = os.path.join(subdir, hyp_files[0])
            subj_str = prefix[3:5] if subset == "sleep-cassette" else prefix[3:5]
            subj_id = int(subj_str)

            Xe,Xo, Xm,labs,subs = load_and_segment(psg_path, hyp_path, subj_id)
            all_Xe.append(Xe);all_Xo.append(Xo);all_Xm.append(Xm);all_y.append(labs);all_subj.append(subs)

X_eeg, X_eog,X_emg, y, subjects = np.vstack(all_Xe),np.vstack(all_Xo),np.vstack(all_Xm),np.concatenate(all_y),np.concatenate(all_subj)

print(f"Total epochs: {len(y)} (EEG shape: {X_eeg.shape}, EOG shape: {X_eog.shape}, EMG shape: {X_emg.shape})")
print(f"class distrib: {np.bincount(y)}")
print(f"# of subjects: {len(set(subjects))}")

/tmp/ipython-input-6-4015245816.py:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(path_psg, preload=True, stim_channel=None, verbose=False)
/tmp/ipython-input-6-4015245816.py:2: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(path_psg, preload=True, stim_channel=None, verbose=False)
/tmp/ipython-input-6-4015245816.py:2: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(path_psg, preload=True, stim_channel=None, verbose=False)
/tmp/ipython-input-6-4015245816.py:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(path_psg, preload=True, stim_channel=None, verbose=False)
/tmp/ipython-input-6-4015245816.py:2: RuntimeWarning: Channels contain different

Total epochs: 28159 (EEG shape: (28159, 3000, 2), EOG shape: (28159, 3000, 1), EMG shape: (28159, 3000, 1))
class distrib: [4461 7292 8171 6282 1953]
# of subjects: 78


In [9]:
!pip install -U scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 108.6 MB/s eta 0:00:00


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
unique_subj = np.unique(subjects)
train_subj,test_subj = train_test_split(np.unique(subjects), test_size=0.2, random_state=42)

train_mask = np.isin(subjects, train_subj)
test_mask = np.isin(subjects, test_subj)

Xe_train, Xe_test, Xo_train, Xo_test, Xm_train,Xm_test,y_train, y_test = X_eeg[train_mask], X_eeg[test_mask], X_eog[train_mask], X_eog[test_mask], X_emg[train_mask], X_emg[test_mask], y[train_mask], y[test_mask]

print(f"Train subj: {len(train_subj)},test subj: {len(test_subj)}")
print(f"Train epochs: {len(y_train)}, Test epochs: {len(y_test)}")
print(f"Train class #: {np.bincount(y_train)}")

Train subj: 62,test subj: 16
Train epochs: 22520, Test epochs: 5639
Train class #: [3526 5800 6584 5046 1564]


In [12]:
!pip install tensorflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00


In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D,BatchNormalization,Activation,MaxPooling1D,GlobalAveragePooling1D,Dense,concatenate

In [14]:
n_eeg, n_eog,n_emg = Xe_train.shape[2], Xo_train.shape[2], Xm_train.shape[2]
input_eeg,input_eog,input_emg = Input(shape=(3000, n_eeg)), Input(shape=(3000, n_eog)), Input(shape=(3000, n_emg))

def build_branch(input_tensor,filters =16):
    x = Conv1D(filters, 64, padding='same')(input_tensor)
    x = BatchNormalization()(x); x = Activation('relu')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(filters*2, 32, padding='same')(x)
    x = BatchNormalization()(x); x = Activation('relu')(x)
    x = MaxPooling1D(4)(x)
    x =GlobalAveragePooling1D()(x)
    return x

branch_eeg,branch_eog,branch_emg = build_branch(input_eeg, filters=16), build_branch(input_eog, filters=8), build_branch(input_emg, filters=8)
merged = concatenate([branch_eeg, branch_eog, branch_emg])
output = Dense(64, activation='softmax')(merged)
output = Dense(5, activation='softmax')(output)

model = Model(inputs=[input_eeg, input_eog, input_emg], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 3000, 2)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3000, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 3000, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 3000, 16)  │      2,064 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 3000, 8)   │        520 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 3000, 8)   │        520 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 3000, 16)  │         64 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3000, 8)   │         32 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3000, 8)   │         32 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 3000, 16)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 3000, 8)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 3000, 8)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 750, 16)   │          0 │ activation[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 750, 8)    │          0 │ activation_2[0][… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 750, 8)    │          0 │ activation_4[0][… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 750, 32)   │     16,416 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 750, 16)   │      4,112 │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 750, 16)   │      4,112 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 750, 32)   │        128 │ conv1d_1[0][0]  

 Total params: 32,613 (127.39 KB)

 Trainable params: 32,421 (126.64 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
perm = np.random.RandomState(0).permutation(len(y_train))
Xe_train, Xo_train, Xm_train, y_train = Xe_train[perm], Xo_train[perm], Xm_train[perm], y_train[perm]
history = model.fit(
    [Xe_train,Xo_train,Xm_train], y_train,
    epochs=20,batch_size=64,validation_split=0.1, verbose=2
)

Epoch 1/20
317/317 - 245s - 772ms/step - accuracy: 0.4329 - loss: 1.4536 - val_accuracy: 0.2695 - val_loss: 1.6066
Epoch 2/20
317/317 - 239s - 755ms/step - accuracy: 0.5321 - loss: 1.2342 - val_accuracy: 0.1585 - val_loss: 1.6346
Epoch 3/20
317/317 - 238s - 751ms/step - accuracy: 0.5745 - loss: 1.1249 - val_accuracy: 0.1581 - val_loss: 1.7614
Epoch 4/20
317/317 - 240s - 757ms/step - accuracy: 0.5999 - loss: 1.0487 - val_accuracy: 0.2695 - val_loss: 1.9779
Epoch 5/20
317/317 - 242s - 763ms/step - accuracy: 0.6162 - loss: 0.9939 - val_accuracy: 0.1585 - val_loss: 2.0877
Epoch 6/20
317/317 - 243s - 767ms/step - accuracy: 0.6254 - loss: 0.9556 - val_accuracy: 0.1585 - val_loss: 2.2288
Epoch 7/20
317/317 - 242s - 762ms/step - accuracy: 0.6270 - loss: 0.9322 - val_accuracy: 0.1585 - val_loss: 2.3182
Epoch 8/20
317/317 - 241s - 760ms/step - accuracy: 0.6336 - loss: 0.9110 - val_accuracy: 0.1585 - val_loss: 2.4172
Epoch 9/20
317/317 - 242s - 762ms/step - accuracy: 0.6342 - loss: 0.9002 - val_a

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
y_pred = model.predict([Xe_test,Xo_test,Xm_test]).argmax(axis=1)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Test accuracy: {acc:.4f}")
print("Confusion Matrix: (rows = true, cols=predicted:\n)",cm)

In [ ]:
Xo_test_zero = np.zeros_like(Xo_test)
y_pred_no_eog = model.predict([Xe_test,Xo_test_zero,Xm_test]).argmax(axis=1)
acc_no_eog = accuracy_score(y_test, y_pred_no_eog)
print(f"Test accuracy without EOG: {acc_no_eog:.4f}")
cm_no_eog = confusion_matrix(y_test, y_pred_no_eog)
print("Confusion Matrix without EOG:\n)",cm_no_eog)